## General Setup

In [ ]:
import glob, os, random

import cv2
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 15]

import keras
from keras.applications import mobilenet_v2, imagenet_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

import tensorflow

In [ ]:
np.random.seed(4096)
tensorflow.set_random_seed(4096)

In [ ]:
WORKING_DIR = '../../../Workspace/bad-cat'
IMAGES_DIR = os.path.join(WORKING_DIR, 'train_images')
OUTPUT_DIR = os.path.join(WORKING_DIR, 'out')

os.makedirs(OUTPUT_DIR, exist_ok=True)

KERAS_MODEL_FILE = os.path.join(OUTPUT_DIR, 'mobilenet_classifier.h5')
TFLITE_MODEL_FILE = os.path.join(OUTPUT_DIR, 'mobilenet_classifier.tflite')
LABELS_FILE = os.path.join(OUTPUT_DIR, 'mobilenet_labels.pickle')

## Setup Image Generators

In [ ]:
IMAGE_W = 224
IMAGE_H = 224

BATCH_SIZE = 32

In [ ]:
image_generator = ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=mobilenet_v2.preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

training_generator = image_generator.flow_from_directory(
    IMAGES_DIR,
    target_size=(IMAGE_W, IMAGE_H),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = image_generator.flow_from_directory(
    IMAGES_DIR,
    target_size=(IMAGE_W, IMAGE_H),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

In [ ]:
labels = [item[0] for item in sorted(training_generator.class_indices.items(), key=lambda item: item[1])]

import pickle
with open(LABELS_FILE, 'wb') as f:
    pickle.dump(labels, f)

## Display Training Data

In [ ]:
def plot_samples(image_dir, grid_width, grid_height):
    image_paths = glob.glob(IMAGES_DIR + '/*/*.jpg')
    random.shuffle(image_paths)
    
    fig, axes = plt.subplots(grid_height, grid_width, subplot_kw={'xticks': [], 'yticks': []})
    fig.set_size_inches(20, 30)
    
    count = grid_width * grid_height
    for ax, path in zip(axes.flat, image_paths[0:count]):
        base, _ = os.path.split(path)
        category = os.path.basename(base)
        image = cv2.imread(path)
        ax.imshow(image)
        ax.set_title(category)

plot_samples(IMAGES_DIR, 6, 12)

## Define Model

In [ ]:
mobilenet_model = mobilenet_v2.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_W,IMAGE_H,3)
)

In [ ]:
for layer in mobilenet_model.layers:
    layer.trainable = False

x = mobilenet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(training_generator.num_classes, activation='softmax')(x)

model = Model(inputs=mobilenet_model.input, outputs=x)

In [ ]:
model.summary()

## Train Model

In [ ]:
TOP_EPOCHS = 10
FINETUNE_EPOCHS = 10

INITIAL_LR = 0.001
FINETUNE_LR = INITIAL_LR / 10.0

In [ ]:
def train(epochs, learning_rate):
    model.compile(
        loss='categorical_crossentropy',
        optimizer = SGD(lr=learning_rate),
        metrics=['accuracy']
    )
    
    return model.fit_generator(
        training_generator,
        validation_data=validation_generator,
        steps_per_epoch=training_generator.samples / training_generator.batch_size,
        validation_steps=validation_generator.samples / validation_generator.batch_size,
        epochs=epochs,
        callbacks=[
            ModelCheckpoint(KERAS_MODEL_FILE, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
        ]
    )

### Train Top Layers

In [ ]:
top_hist = train(TOP_EPOCHS, INITIAL_LR)

### Fine Tune Model

In [ ]:
for layer in mobilenet_model.layers[:100]:
    layer.trainable = True

finetune_hist = train(FINETUNE_EPOCHS, FINETUNE_LR)

## Export as TFLite Model

In [ ]:
converter = tensorflow.contrib.lite.TFLiteConverter.from_keras_model_file(KERAS_MODEL_FILE)
converted_model = converter.convert()
with open(TFLITE_MODEL_FILE, 'wb') as f:
    f.write(converted_model)
    
keras.backend.clear_session()

## Evaluate Training Results

In [ ]:
import matplotlib.pyplot as plt

top_hist = finetune_hist = H
N = np.arange(0, TOP_EPOCHS + FINETUNE_EPOCHS)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,8))

ax1.plot(N, top_hist.history["loss"] + finetune_hist.history["loss"], label="Training Loss")
ax1.plot(N, top_hist.history["val_loss"] + finetune_hist.history["val_loss"], label="Validation Loss")
ax1.set(title="Loss", xlabel='Epoch', ylabel='Loss')
ax1.legend()

ax2.plot(N, top_hist.history["acc"] + finetune_hist.history["acc"], label="Training Accuracy")
ax2.plot(N, top_hist.history["val_acc"] + finetune_hist.history["val_acc"], label="Validation Accuracy")
ax2.set(title='Accuracy', xlabel='Epoch', ylabel='Accuracy', ylim=(0,1))
ax2.legend()

plt.show()

In [ ]:
model = keras.models.load_model(KERAS_MODEL_FILE)

predictions = model.predict_generator(validation_generator, steps=validation_generator.samples / validation_generator.batch_size)
predictions = np.argmax(predictions, axis=1)

true_classes = validation_generator.classes
class_labels = validation_generator.class_indices.keys()

import sklearn.metrics as metrics
print(metrics.confusion_matrix(y_true=true_classes, y_pred=predictions))
print(metrics.classification_report(true_classes, predictions))